In [66]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name

bucket='sumalr' # Replace with your s3 bucket name
prefix = 'data' # Used as part of the path in the bucket where you store data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket) # The URL to access the bucket
bucket_path

'https://s3-us-east-1.amazonaws.com/sumalr'

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

<pre>X = np.linspace(1,700,257)
with open('lrdata.csv','w') as f:
    y = lambda x:1.5*x-1.2
    f.write('x,y\n')
    for i in X:
        f.write(str(i)+','+str(y(i))+'\n')</pre>

In [68]:
data = pd.read_csv('s3://sumalr/lrdata.csv')
data.head()

,x,y
0,1.000000,0.300000
1,3.730469,4.395703
2,6.460938,8.491406
3,9.191406,12.587109
4,11.921875,16.682813


In [69]:
from sklearn.model_selection  import train_test_split
train_X,test_X,train_y,test_y = train_test_split(data[['x']],data['y'])

In [70]:
train = np.array(train_X.iloc[:, :]).astype('float32')
# Actual Fantasy Points
train_target = np.array(train_y).astype('float32')
#Verify that the conversion worked
print(train[0],train_target[0])

[476.10156] 712.95233


In [71]:
test = np.array(test_X.iloc[:, :]).astype('float32')
# Actual Fantasy Points
test_target = np.array(test_y).astype('float32')
#Verify that the conversion worked
print(test[0],test_target[0])
print(test_target.shape)

[514.3281] 770.2922
(65,)


In [72]:
import sagemaker.amazon.common as smac

In [73]:
print('train_features shape = ', train.shape)
print('train_labels shape = ', train_target.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train, train_target)
buf.seek(0)

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

train_features shape =  (192, 1)
train_labels shape =  (192,)
uploaded training data location: s3://sumalr/data/train/protobuf_data


In [74]:
print('test_features shape = ', test.shape)
print('test_labels shape = ', test_target.shape)
key = 'protobuf_data'

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test, test_target)
buf.seek(0)

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded test data location: {}'.format(s3_test_data))

test_features shape =  (65, 1)
test_labels shape =  (65,)
uploaded test data location: s3://sumalr/data/test/protobuf_data


In [75]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [76]:
output_path = 's3://' + bucket + '/' + 'default_example/output'

In [77]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_path,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=1,
                           predictor_type='regressor',
                           mini_batch_size=5)

linear.fit({'train': s3_train_data})

2019-06-30 10:34:36 Starting - Starting the training job...
2019-06-30 10:34:39 Starting - Launching requested ML instances......
2019-06-30 10:35:40 Starting - Preparing the instances for training......
2019-06-30 10:36:48 Downloading - Downloading input data...
2019-06-30 10:37:25 Training - Training image download completed. Training in progress.
Docker entrypoint called with argument(s): train
[06/30/2019 10:37:27 INFO 139754587887424] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u

[2019-06-30 10:37:32.686] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 14, "duration": 863, "num_examples": 39, "num_bytes": 9216}
#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.08812896750475231, "sum": 0.08812896750475231, "min": 0.08812896750475231}}, "EndTime": 1561891052.686837, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1561891052.686746}

#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.07454122963704561, "sum": 0.07454122963704561, "min": 0.07454122963704561}}, "EndTime": 1561891052.686929, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1561891052.68691}

#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.08942173350798456, "sum": 0.08942173350798456, "min": 0.08942173350798456}}, "EndTime": 1561891052.686991, "Dimensions": 


2019-06-30 10:37:47 Uploading - Uploading generated training model
2019-06-30 10:37:47 Completed - Training job completed
Billable seconds: 60


In [78]:
instance_type = 'ml.m4.xlarge'
model_name = 'linear_%s'% instance_type
endpoint_name = 'linear-ml-m4-xlarge-%s'% (str(time.time()).replace('.','-'))
print('setting up the endpoint..')
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.t2.medium',endpoint_name=endpoint_name)


setting up the endpoint..
--------------------------------------------------------------------------------------------------------------------------!

In [57]:
from sagemaker.predictor import csv_serializer, json_deserializer
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [79]:
endpoint_name

'linear-ml-m4-xlarge-1561909713-7209613'

In [80]:
test[0]

array([514.3281], dtype=float32)

In [61]:
result = linear_predictor.predict(test[0])
print(result)

{'predictions': [{'score': 114.9801254272461}]}


In [62]:
test_target[0]

114.97969

In [ ]:
def delete_endpoint(predictor):
    try:
        boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
        print('Deleted {}'.format(predictor.endpoint))
    except:
        print('Already deleted: {}'.format(predictor.endpoint))

delete_endpoint(predictor)